# Analyse der generierten Pressemitteilungen

Dieses Notebook dient der Analyse der mit dem Teuken-Modell generierten Pressemitteilungen. Wir vergleichen die generierten Texte mit den Original-Pressemitteilungen und untersuchen verschiedene Aspekte wie Länge, Struktur und Qualität.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Setze den Stil für bessere Visualisierungen
plt.style.use('seaborn')
sns.set_palette("husl")

# Konfiguriere die Anzeige von Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

## 1. Daten laden und vorbereiten

In [ ]:
# Lade die generierten Pressemitteilungen
generated_df = pd.read_csv('data/generation/teuken-7b_results.csv')

# Lade den Original-Datensatz für den Vergleich
original_df = pd.read_csv('data/processed/cases_prs_synth_prompts_subset.csv')

# Zeige die ersten Zeilen
print("Generierte Pressemitteilungen:")
display(generated_df.head())
print("\nOriginal-Datensatz:")
display(original_df.head())

## 2. Grundlegende Statistiken

In [ ]:
def calculate_text_stats(text):
    """Berechnet grundlegende Statistiken für einen Text."""
    if pd.isna(text):
        return pd.Series({'length': 0, 'words': 0, 'sentences': 0})
    
    # Berechne Länge in Zeichen
    length = len(str(text))
    
    # Berechne Anzahl der Wörter
    words = len(str(text).split())
    
    # Berechne Anzahl der Sätze (einfache Approximation)
    sentences = len([s for s in str(text).split('.') if s.strip()])
    
    return pd.Series({'length': length, 'words': words, 'sentences': sentences})

# Berechne Statistiken für beide Datensätze
generated_stats = generated_df['generated_pr_teuken'].apply(calculate_text_stats)
original_stats = original_df['summary'].apply(calculate_text_stats)

# Zeige Zusammenfassung
print("Statistiken für generierte Pressemitteilungen:")
display(generated_stats.describe())
print("\nStatistiken für Original-Pressemitteilungen:")
display(original_stats.describe())

## 3. Visualisierung der Textlängen

In [ ]:
plt.figure(figsize=(12, 6))

# Erstelle Boxplots für beide Datensätze
data = [generated_stats['words'], original_stats['words']]
plt.boxplot(data, labels=['Generiert', 'Original'])

plt.title('Vergleich der Wortanzahl')
plt.ylabel('Anzahl der Wörter')
plt.grid(True, alpha=0.3)

plt.show()

## 4. Qualitative Analyse

In [ ]:
def display_comparison(idx):
    """Zeigt einen Vergleich zwischen generierter und Original-Pressemitteilung."""
    print(f"\nVergleich für Eintrag {idx}:")
    print("\nGenerierte Pressemitteilung:")
    print(generated_df.iloc[idx]['generated_pr_teuken'])
    print("\nOriginal-Pressemitteilung:")
    print(original_df.iloc[idx]['summary'])
    print("\n" + "="*80)

# Zeige einige Beispiele
for idx in range(min(3, len(generated_df))):
    display_comparison(idx)

## 5. Strukturanalyse

In [ ]:
def analyze_structure(text):
    """Analysiert die Struktur eines Textes."""
    if pd.isna(text):
        return pd.Series({
            'has_date': False,
            'has_location': False,
            'has_quotes': False,
            'starts_with_date': False
        })
    
    text = str(text)
    
    # Prüfe auf Datum (einfache Approximation)
    has_date = any(char.isdigit() for char in text[:50])
    
    # Prüfe auf Ortsangaben (einfache Approximation)
    has_location = any(word in text.lower() for word in ['stadt', 'land', 'bundesland', 'gericht'])
    
    # Prüfe auf Anführungszeichen
    has_quotes = '"' in text or '\'\'' in text
    
    # Prüfe, ob der Text mit einem Datum beginnt
    starts_with_date = text[0].isdigit() if text else False
    
    return pd.Series({
        'has_date': has_date,
        'has_location': has_location,
        'has_quotes': has_quotes,
        'starts_with_date': starts_with_date
    })

# Analysiere die Struktur beider Datensätze
generated_structure = generated_df['generated_pr_teuken'].apply(analyze_structure)
original_structure = original_df['summary'].apply(analyze_structure)

# Zeige die Ergebnisse
print("Strukturanalyse der generierten Pressemitteilungen:")
display(generated_structure.mean())
print("\nStrukturanalyse der Original-Pressemitteilungen:")
display(original_structure.mean())

## 6. Zusammenfassung und Erkenntnisse

In [ ]:
# Erstelle eine Zusammenfassung der wichtigsten Erkenntnisse
print("Zusammenfassung der Analyse:")
print(f"\n1. Durchschnittliche Länge:")
print(f"   - Generiert: {generated_stats['words'].mean():.1f} Wörter")
print(f"   - Original: {original_stats['words'].mean():.1f} Wörter")

print(f"\n2. Strukturmerkmale:")
print(f"   - Datum enthalten:")
print(f"     * Generiert: {generated_structure['has_date'].mean()*100:.1f}%")
print(f"     * Original: {original_structure['has_date'].mean()*100:.1f}%")

print(f"   - Ortsangaben enthalten:")
print(f"     * Generiert: {generated_structure['has_location'].mean()*100:.1f}%")
print(f"     * Original: {original_structure['has_location'].mean()*100:.1f}%")